In [101]:
from scipy import signal, linalg
from scipy import linalg
import scipy.io.wavfile
import numpy as np
import os


path_db = '05_AudioClassifier_Pdf/05_AudioClassifier_Pdf/database/'
path_music = path_db+'music/'
path_speech = path_db+'speech/'
path_unknowns = path_db.replace('database/','')+'unknownSounds/'

spg_music = 0
spg_speech = 0


def load_audio(path):
    audio_list = []#in py questa è una semplice lista vuota
    for filename in os.listdir(path):
        audio_list.append((scipy.io.wavfile.read(path+filename)[0],scipy.io.wavfile.read(path+filename)[1],filename))
    return audio_list
    
def spg(path):
    start = True
    _spg = 0
    for filename in os.listdir(path):
        rate,data = scipy.io.wavfile.read(path+filename)
        f, t, Sxx = signal.spectrogram(data, rate, 'hamming',512,160,512)
        if start:
            _spg = Sxx
            start=False
        else:
            _spg = scipy.hstack((_spg,Sxx))
    return _spg

def spg_b(s,V):
    #s è il vettore(sarebbe matrice ma diagonale) dei valori singolari
    #V è la matrice dei vettori destri (la funzione vuole V.T=>autovettori per colonna)
    ssum = np.sum(s)
    approx_sum = 0
    i = 0;
    for e_value in s:
        approx_sum = approx_sum + e_value
        if approx_sum >= ssum*0.9:
            break
        i = i+1
    base = V[:,range(0,i)]
    return base
    

In [102]:

spg_music = spg(path_music).T
print("spg_music shape: ",spg_music.shape)
spg_music = np.delete(spg_music,256,0)
U, s, V= np.linalg.svd(spg_music)
b_music = spg_b(s,V)
print("b_music shape",b_music.shape)
norm_music = np.dot(b_music.conj().T,b_music)
print("norm_music shape",norm_music.shape)


spg_music shape:  (9080, 257)
b_music shape (257, 68)
norm_music shape (68, 68)


In [103]:

spg_speech = spg(path_speech).T
print("spg_speech shape: ",spg_speech.shape)
spg_speech = np.delete(spg_speech,256,0)
U, s, V= np.linalg.svd(spg_speech)
b_speech = spg_b(s,V)
print("b_speech shape",b_speech.shape)
norm_speech = np.dot(b_speech.conj().T,b_speech)
print("norm_speech shape",norm_speech.shape)

spg_speech shape:  (2980, 257)
b_speech shape (257, 39)
norm_speech shape (39, 39)


In [104]:
unknowns = load_audio(path_unknowns)
unknowns

[(16000,
  array([ 1520,  1164,  1560, ...,  -232,  -940, -1979], dtype=int16),
  '004_BobDylan-OxfordTown-10s-B.wav'),
 (16000, array([17, 24, 15, ...,  5,  7, -1], dtype=int16), '007si1079.wav'),
 (16000, array([1, 2, 2, ..., 1, 0, 5], dtype=int16), '007si1271.wav'),
 (16000,
  array([-1617, -1563, -1473, ..., -3358, -3364, -2902], dtype=int16),
  '010_CiboMatto-Moonchild-10s-A.wav'),
 (16000, array([2, 3, 2, ..., 3, 0, 0], dtype=int16), '014si1291.wav'),
 (16000, array([2, 3, 3, ..., 6, 6, 6], dtype=int16), '016si1621.wav'),
 (16000,
  array([ 1424,  6801,  8511, ..., -2669,  1359, -1264], dtype=int16),
  '019_Nirvana-Downer-10s-B.wav'),
 (16000,
  array([-2609, -2725, -2910, ...,  3971,  4190,  4345], dtype=int16),
  '024_JimiHendrix-WaitUntilTomorrow-10s-A.wav')]

In [105]:
for rate,data,filename in unknowns:
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    spg_unk = Sxx
    print('Item: ',filename)
    print('\tOnSpeech: ',Sxx.T.shape,'X',b_speech.shape,'=',np.dot(Sxx.T,b_speech).shape) 
    print('\tOnMusic: ',Sxx.T.shape,'X',b_music.shape,'=',np.dot(Sxx.T,b_music).shape) 
    unk_on_music = np.dot(Sxx.T,b_music)
    unk_on_speech = np.dot(Sxx.T,b_speech)
    
    coeff_on_music = np.dot(np.linalg.inv(norm_music),unk_on_music.T)
    spg_ric_on_music = np.dot(coeff_on_music.T,b_music.T).T
    error_on_music = np.linalg.norm(spg_unk-spg_ric_on_music)
    
    coeff_on_speech = np.dot(np.linalg.inv(norm_speech),unk_on_speech.T)
    spg_ric_on_speech = np.dot(coeff_on_speech.T,b_speech.T).T
    error_on_speech = np.linalg.norm(spg_unk-spg_ric_on_speech)
    
    print('\terr on music',error_on_music)
    print('\terr on speech',error_on_speech)
    if error_on_music<error_on_speech: print('\t\t\t\tMUSICA')
    else: print('\t\t\t\tPARLATO')
    
#NON FUNZIONA PER NULLA

Item:  004_BobDylan-OxfordTown-10s-B.wav
	OnSpeech:  (454, 257) X (257, 39) = (454, 39)
	OnMusic:  (454, 257) X (257, 68) = (454, 68)
	err on music 429881.0
	err on speech 1.33751e+06
				MUSICA
Item:  007si1079.wav
	OnSpeech:  (210, 257) X (257, 39) = (210, 39)
	OnMusic:  (210, 257) X (257, 68) = (210, 68)
	err on music 14765.2
	err on speech 44889.5
				MUSICA
Item:  007si1271.wav
	OnSpeech:  (220, 257) X (257, 39) = (220, 39)
	OnMusic:  (220, 257) X (257, 68) = (220, 68)
	err on music 8610.82
	err on speech 22084.4
				MUSICA
Item:  010_CiboMatto-Moonchild-10s-A.wav
	OnSpeech:  (454, 257) X (257, 39) = (454, 39)
	OnMusic:  (454, 257) X (257, 68) = (454, 68)
	err on music 379715.0
	err on speech 1.53214e+06
				MUSICA
Item:  014si1291.wav
	OnSpeech:  (214, 257) X (257, 39) = (214, 39)
	OnMusic:  (214, 257) X (257, 68) = (214, 68)
	err on music 52249.8
	err on speech 115502.0
				MUSICA
Item:  016si1621.wav
	OnSpeech:  (71, 257) X (257, 39) = (71, 39)
	OnMusic:  (71, 257) X (257, 68) 

In [112]:
music = load_audio(path_music)
speech = load_audio(path_speech)

In [115]:
media = 0
i = 0
for rate,data,filename in music:
    i+=1
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    spg_unk = Sxx
    unk_on_music = np.dot(Sxx.T,b_music)
    
    coeff_on_music = np.dot(np.linalg.inv(norm_music),unk_on_music.T)
    spg_ric_on_music = np.dot(coeff_on_music.T,b_music.T).T
    
    media += np.mean(np.mean(spg_ric_on_music,0),0)
media_m = media/i


media = 0
i = 0
for rate,data,filename in speech:
    i+=1
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    spg_unk = Sxx
    unk_on_speech = np.dot(Sxx.T,b_speech)
    
    coeff_on_speech = np.dot(np.linalg.inv(norm_speech),unk_on_speech.T)
    spg_ric_on_speech = np.dot(coeff_on_speech.T,b_speech.T).T
    
    media += np.mean(np.mean(spg_ric_on_speech,0),0)
    media_s = media/i
    
print('mean speech',media_s)
print('mean music',media_m)


mean speech 16.9067526221
mean music 1844.90585938


In [120]:
for rate,data,filename in unknowns:
    f, t, Sxx = signal.spectrogram(data , rate, 'hamming',512,160,512)
    spg_unk = Sxx
    print('Item: ',filename)
    print('\tOnSpeech: ',Sxx.T.shape,'X',b_speech.shape,'=',np.dot(Sxx.T,b_speech).shape) 
    print('\tOnMusic: ',Sxx.T.shape,'X',b_music.shape,'=',np.dot(Sxx.T,b_music).shape) 
    unk_on_music = np.dot(Sxx.T,b_music)
    unk_on_speech = np.dot(Sxx.T,b_speech)
    
    coeff_on_music = np.dot(np.linalg.inv(norm_music),unk_on_music.T)
    spg_ric_on_music = np.dot(coeff_on_music.T,b_music.T).T
    mean_on_m = np.mean(np.mean(spg_ric_on_music,0),0)
    coeff_on_speech = np.dot(np.linalg.inv(norm_speech),unk_on_speech.T)
    spg_ric_on_speech = np.dot(coeff_on_speech.T,b_speech.T).T
    mean_on_s = np.mean(np.mean(spg_ric_on_speech,0),0)
    
    err_on_music = np.abs(media_m-mean_on_m)
    err_on_speech = np.abs(media_s-mean_on_s)
    print('\terr on music',err_on_music)
    print('\terr on speech',err_on_speech)
    if err_on_music<err_on_speech: print('\t\t\t\tMUSICA')
    else: print('\t\t\t\tPARLATO')
    
#QUASI FUNZIONA

Item:  004_BobDylan-OxfordTown-10s-B.wav
	OnSpeech:  (454, 257) X (257, 39) = (454, 39)
	OnMusic:  (454, 257) X (257, 68) = (454, 68)
	err on music 879.857275391
	err on speech 820.856308901
				PARLATO
Item:  007si1079.wav
	OnSpeech:  (210, 257) X (257, 39) = (210, 39)
	OnMusic:  (210, 257) X (257, 68) = (210, 68)
	err on music 1814.19561615
	err on speech 6.74200149775
				PARLATO
Item:  007si1271.wav
	OnSpeech:  (220, 257) X (257, 39) = (220, 39)
	OnMusic:  (220, 257) X (257, 68) = (220, 68)
	err on music 1835.01966419
	err on speech 8.84447387457
				PARLATO
Item:  010_CiboMatto-Moonchild-10s-A.wav
	OnSpeech:  (454, 257) X (257, 39) = (454, 39)
	OnMusic:  (454, 257) X (257, 68) = (454, 68)
	err on music 210.910791016
	err on speech 2020.96910187
				MUSICA
Item:  014si1291.wav
	OnSpeech:  (214, 257) X (257, 39) = (214, 39)
	OnMusic:  (214, 257) X (257, 68) = (214, 68)
	err on music 1790.34104385
	err on speech 23.933064425
				PARLATO
Item:  016si1621.wav
	OnSpeech:  (71, 257) X (2